In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence

In [ ]:

train = "D:\\data_yaaay\\waaa\\cX10Copy\\train"
test = "D:\\data_yaaay\\waaa\\cX10Copy\\test"
df = pd.read_excel("D:\\data_yaaay\\KARSL-190_Labels.xlsx")


In [25]:
print(df.head())
print(f"Columns: {df.columns.tolist()}")
print(f"Number of classes: {len(df)}")

   SignID Sign-Arabic Sign-English
0       1           0            0
1       2           1            1
2       3           2            2
3       4           3            3
4       5           4            4
Columns: ['SignID', 'Sign-Arabic', 'Sign-English']
Number of classes: 190


In [ ]:
def load_images_with_labels(base_path, labels_df, frames_per_session=5, max_sessions_per_class=2):
    images = []
    labels = []
    sign_ids = []
    total_loaded = 0
    
    for class_folder in os.listdir(base_path):
        class_path = os.path.join(base_path, class_folder)
        if os.path.isdir(class_path):
            # Extract SignID from folder name
            try:
                sign_id = int(class_folder)
            except ValueError:
                print(f"Skipping folder {class_folder} - not a valid SignID")
                continue
            
            # Get label from Excel file
            sign_row = labels_df[labels_df['SignID'] == sign_id]
            if sign_row.empty:
                print(f"No label found for SignID {sign_id}")
                continue
            
            sign_english = sign_row['Sign-English'].iloc[0]
            
            # Limit sessions per class to manage memory
            session_folders = [f for f in os.listdir(class_path) 
                             if os.path.isdir(os.path.join(class_path, f))]
            session_folders = session_folders[:max_sessions_per_class]  # Limit sessions
            
            for session_folder in session_folders:
                session_path = os.path.join(class_path, session_folder)
                
                # Get all frames and sort them (maintain sequence order)
                frame_files = [f for f in os.listdir(session_path) 
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                frame_files.sort()
                
                if len(frame_files) == 0:
                    continue
                
                # Sample frames evenly across the sequence to maintain temporal flow
                if len(frame_files) <= frames_per_session:
                    selected_frames = frame_files
                else:
                    # Sample evenly to keep sequence information
                    indices = np.linspace(0, len(frame_files)-1, frames_per_session, dtype=int)
                    selected_frames = [frame_files[i] for i in indices]
                
                # Load selected frames
                for filename in selected_frames:
                    img_path = os.path.join(session_path, filename)
                    img = cv2.imread(img_path)
                    if img is not None:
                        # Use smaller size to save memory
                        img = cv2.resize(img, (128, 128))  # Reduced from 256x256
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = img.astype(np.float32) / 255.0  # Use float32 instead of float64
                        
                        images.append(img)
                        labels.append(sign_english)
                        sign_ids.append(sign_id)
                        total_loaded += 1
                        
                        # Print progress every 1000 images
                        if total_loaded % 1000 == 0:
                            print(f"Loaded {total_loaded} images...")
            
            # Print progress per class
            class_images = len([l for l in labels if l == sign_english])
            print(f"SignID {sign_id} ({sign_english}): {class_images} images from {len(session_folders)} sessions")
    
    print(f"Total images loaded: {total_loaded}")
    return np.array(images, dtype=np.float32), np.array(labels), np.array(sign_ids)

In [ ]:
print("Loading training data with labels...")
X_train, y_train_labels, train_sign_ids = load_images_with_labels(
    train, df, 
    frames_per_session=3,      
    max_sessions_per_class=2   
)

print("Loading test data with labels...")
X_test, y_test_labels, test_sign_ids = load_images_with_labels(
    test, df,
    frames_per_session=3,
    max_sessions_per_class=2
)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_labels)
y_test_encoded = label_encoder.transform(y_test_labels)

# Convert to categorical
num_classes = len(label_encoder.classes_)
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded, num_classes)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded, num_classes)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Number of classes: {num_classes}")
print(f"Sample labels: {label_encoder.classes_[:10]}")  # Show first 10 class names

# Create a mapping for reference
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(f"Label mapping (first 5): {dict(list(label_mapping.items())[:5])}")

In [ ]:
def reshape_to_sequences(X, y, frames_per_sequence=3):
    sequences = []
    labels = []
    for i in range(0, len(X), frames_per_sequence):
        if i + frames_per_sequence <= len(X):
            sequence = X[i:i+frames_per_sequence]
            sequences.append(sequence)
            labels.append(y[i])
    return np.array(sequences), np.array(labels)

X_train_seq, y_train_seq = reshape_to_sequences(X_train, y_train_categorical, 3)
X_test_seq, y_test_seq = reshape_to_sequences(X_test, y_test_categorical, 3)

In [ ]:
def create_resnet_bigru_model(input_shape, num_classes):
    # ResNet50 
    base_resnet = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(128, 128, 3)
    )
    base_resnet.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    
    # Extract features from each frame
    x = tf.keras.layers.TimeDistributed(base_resnet)(inputs)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAveragePooling2D())(x)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
    
    # Bidirectional GRU
    x = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(256, return_sequences=True, dropout=0.5)
    )(x)
    x = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(128, dropout=0.5)
    )(x)
    
    # Classification 
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
model_resnet_bigru = create_resnet_bigru_model((3, 128, 128, 3), num_classes)

model_resnet_bigru.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'top_5_accuracy']
)


model_resnet_bigru.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=15, 
        restore_best_weights=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.5, 
        patience=7,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_sign_model.h5', 
        save_best_only=True,
        monitor='val_accuracy'
    )
]

history = model_resnet_bigru.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

def plot_simple_results(history):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Training & Validation Accuracy
    axes[0].plot(history.history['accuracy'], label='Training Accuracy', color='blue')
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)
    
    # Training & Validation Loss
    axes[1].plot(history.history['loss'], label='Training Loss', color='blue')
    axes[1].plot(history.history['val_loss'], label='Validation Loss', color='red')
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
   
    print(f"Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print(f"Training Loss: {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")

plot_simple_results(history)

In [ ]:
test_loss, test_acc, test_top5 = model_resnet_bigru.evaluate(X_test_seq, y_test_seq, verbose=0)

print(f"Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f" Top-5 Accuracy: {test_top5:.4f} ({test_top5*100:.2f}%)")
print(f"Loss: {test_loss:.4f}")

# Quick predictions analysis
y_pred = model_resnet_bigru.predict(X_test_seq[:100], verbose=0)  # First 100 samples
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_seq[:100], axis=1)

print(f"\n=== Sample Predictions ===")
for i in range(5):  # Show first 5 predictions
    true_label = label_encoder.classes_[y_true_classes[i]]
    pred_label = label_encoder.classes_[y_pred_classes[i]]
    confidence = y_pred[i].max()
    print(f"Sample {i+1}: True='{true_label}' | Pred='{pred_label}' | Confidence={confidence:.3f}")

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")


model_filename = f'resnet_bigru_model{timestamp}.h5'
model_resnet_bigru.save(model_filename)

print(f"model saved: {model_filename}")